# Robust vehicle pre-allocation

This tutorial shows how to construct and solve the vehicle pre-allocation problem problem (Hao et al., 2020, https://onlinelibrary.wiley.com/doi/full/10.1111/poms.13143?casa_token=QjXxs7wyx8cAAAAA%3ADp6PFQDpob1UUd7qG8mSSlyEwmGjPrcjYaeSZxp8mmKaL1FUPfDVqcW4xt-HHLW_mjZlHIJOoETs2qo6) The problem can be formulated as follows:
$$
\begin{align}
\min_{\mathbf{x}, \mathbf{Y}}\; & \left( \mathbf{c}-\mathbf{r} \right)\mathbf{x} + \mathbf{r}\mathbf{Y} \nonumber \\

\text{s.t. } & \mathbf{0} \leq \mathbf{P} \leq \mathbf{P_{max}} \nonumber \\

&\sum_{t=1}^T \mathbf{p}_t\leq Q_i \nonumber \\

&v_{min} \leq v_0 + \sum_{t'=1}^{t-1}\sum_{i=1}^I p_{i, t'} - \sum_{t'=1}^{t-1} \leq v_{max} \forall t\in \left\{ 1, \ldots, T \right\} \nonumber \\
\end{align}
$$

where $\mathbf{d}\in\mathbb{R}^T$ is the uncertain demand vector, $\mathbf{P}\in\mathbb{R}^{I\times T}$ is resource decision variable, i.e. $y_{i,t}$ there is a prodcut order of $p_{i,t}$ at factory $i$ during time $t$. The variable $\mathbf{P}$ depends on the demand vector $\mathbf{d}$ and can be written as
$$
\mathbf{P} = \mathbf{P}^0 + \sum_{\tau=1}^{t-1}\mathbf{P}^{\tau}d_{\tau}
$$
where $\mathbf{P}^0, \left\{ \mathbf{P}^{\tau} \right\}_\tau \in \mathbb{R}^{I\times T}$, and the $\mathbf{d}$ belongs to the following uncertainty set:
$$
\mathbf{d} \in \left\{ \mathbf{d}: \left( 1-\theta \right)d_t^0 \leq d_t \leq \left( 1+\theta \right)d_t^0  \right\}
$$
and $\mathbf{d^0}\in\mathbb{R}^{T}$ is a known vector.

In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd

from lropt import Ellipsoidal, UncertainParameter, RobustProblem

In [2]:
data = pd.read_csv('https://xiongpengnus.github.io/rsome/taxi_rain.csv')
demand = data.loc[:, 'Region1':'Region10']  
data = demand.values                              # sample demand as an array
d_ub = demand.max().values                      # upper bound of demand
d_lb = demand.min().values 
I, J = 1, 10
r = np.array([4.50, 4.41, 3.61, 4.49, 4.38, 4.58, 4.53, 4.64, 4.58, 4.32])
c = 3 * np.ones(J)
q = 400

In [4]:
x = cp.Variable(J)
y_0 = cp.Variable(J)
y_k = [cp.Variable(J) for _ in range(J)]
uncertainty_set = Ellipsoidal(p=np.inf, rho = np.max(d_ub), lb = d_lb, ub = d_ub)
d = UncertainParameter(J, uncertainty_set=uncertainty_set)
y = 0
for j in range(J):
    y += y_k[j]
y += y_0


objective = cp.Minimize((c-r)@x + r@y)
constraints = [
                y >= x-d,
                y >= 0,
                cp.sum(x) <= q,
                x >= 0,
              ]
prob = RobustProblem(objective=objective, constraints=constraints)
prob.solve()

In [3]:
u = lropt.UncertainParameter(J,
                        uncertainty_set=lropt.Ellipsoidal(p=np.inf, rho = np.max(d_ub),
                                                          lb = d_lb, ub = d_ub))
# Formulate the Robust Problem
x = cp.Variable(J)
y = cp.Variable(J)
Y = cp.Variable((J,J))
objective = cp.Minimize((c-r)@x + r@y + r@Y@u)
constraints = []
for i in range(J):
    constraints += [y[i] >= x[i] - (Y[i]+np.eye(J)[i])@u]
    constraints += [y[i] + Y[i]@u >= 0]
constraints += [x >= 0, cp.sum(x)<= q]
prob = lropt.RobustProblem(objective=objective, constraints=constraints)

In [4]:
prob.solve()

-62.58973526877305